# 12주차

블록체인에서 주문을 하고, 그 주문내역을 로컬 파일에 작성하세요.

블록체인으로 주문이 전송되면 이벤트가 발생하도록 프로그램하고, 주문내역은 로컬파일에 저장됩니다.

* 강의자료 OrderEvent.sol을 참조해서 구현하세요.

	* order(상품항목, 주문개수, 단가, 배송지) 함수를 작성. '개수 x 단가'로 주문금액을 계산하고 이 금액만큼 입금되지 않으면 예외발생.

	* getBalance() 함수를 작성. 최초 프로그램 생성자 owner만 읽을 수 있게 modfier isOwner 작성.

(1) REMIX에서 주문하고, 이벤트 발생하는지 보이도록 콘솔 출력 붙여넣으세요.

(2) 파일로 컴파일 하세요.

(3) ganache에서 배포하세요.

(4) 주문을 3건만 하고, 이벤트를 통해 다음과 같이 출력하세요.

항목은 차례대로 주소, 상품항목번호, 주문개수, 단가, 배송지.

```

전송자주소, 1111, 3, 100, 20 2-gil Hongji-dong Jongro-gu Seoul

전송자주소, 1111, 5, 100, 20 2-gil Hongji-dong Jongro-gu Seoul

전송자주소, 1111, 20, 100, 20 2-gil Hongji-dong Jongro-gu Seoul

```

(5) 파일에 저장된 파일내역을 출력하세요 (주피터 노트북에서 !type src/OrderEvent.txt)

### 1) remix IDE에서 주문

In [ ]:
[vm]from: 0x5B3...eddC4to: OrderEvent.(constructor)value: 0 weidata: 0x608...70033logs: 0hash: 0xc9e...a65a6
status	true Transaction mined and execution succeed
transaction hash	0xc9ea68911de79bf4c52f50ef58b5ea9d45ba8889eaa5d8d7fb16baec3e1a65a6
from	0x5B38Da6a701c568545dCfcB03FcB875f56beddC4
to	OrderEvent.(constructor)
gas	455358 gas
transaction cost	395963 gas 
execution cost	395963 gas 
input	0x608...70033
decoded input	{}
decoded output	 - 
logs	[]
val	0 wei
transact to OrderEvent.order errored: Error encoding arguments: Error: invalid arrayify value (argument="value", value="1111", code=INVALID_ARGUMENT, version=bytes/5.5.0)

###  2) 파일로 컴파일 하세요.

In [2]:
%%writefile src/OrderEvent.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract OrderEvent {
    
    address payable owner;
    event OrderLog(address indexed _from, bytes2 _itemId, uint indexed quantity, uint Price, string add);

    constructor() {
        owner = payable(msg.sender);
    }


    function order(bytes2 _itemId, uint quantity, uint Price, string memory add) public payable isOwner{
        uint256 orderAmount = quantity * Price;
        require(msg.value == orderAmount);
        emit OrderLog(msg.sender, _itemId, quantity, msg.value, add);
    
    }

    modifier isOwner() { 
        require(msg.sender == owner);
        _;
    }
    
    function getBalance() view public isOwner returns(uint) {
        return address(this).balance;
    }

}

Overwriting src/OrderEvent.sol


### 3) ganache에서 배포하세요.



In [3]:
!solc --optimize --combined-json abi,bin src/OrderEvent.sol > src/OrderEvent.json

In [4]:
%%writefile src/OrderEvent.js
var Web3 = require('web3');
var _abiBinJson = require('./OrderEvent.json');  
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiBinJson.contracts);
console.log("- contract name: ", contractName);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
_bin=_abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: "0x"+_bin})
        .send({from: accounts[0], gas: 364124}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/OrderEvent.js


In [5]:
!node src/OrderEvent.js

- contract name:  [ 'src/OrderEvent.sol:OrderEvent' ]
Deploying the contract from 0xE91Bf916E93053e0FE4628365afb1Ca16b92D758
hash: 0xdbff1046096e7f307a4822b07b2812b85b8f37c3d8ff7a8f3a46593ab4ed5aae
---> The contract deployed to: 0x559cE62C8650F47Ed81A661adA90fA6Ed958666D


### 4) 주문을 3건만 하고, 이벤트를 통해 다음과 같이 출력하세요.


In [6]:
%%writefile src/OrderEventUse.js
var Web3=require('web3');
var fs = require('fs');
var _abiBinJson = require('./OrderEvent.json');      //importing a javascript file

var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
contractName=Object.keys(_abiBinJson.contracts); // reading ['src/OrderEvent.sol:OrderEvent']

_abiArray=_abiBinJson.contracts[contractName].abi; //use just as read from file
console.log("- ABI: " + _abiArray);

async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, '0x559cE62C8650F47Ed81A661adA90fA6Ed958666D');
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    var event = _order.events.OrderLog({
            filter: {_from: accounts[0]},
            fromBlock: 'latest', toBlock: 'pending'
        }, function (error, result) {
        if (!error) {
            //log = JSON.stringify(result.returnValues);
            log = result.returnValues
            //console.log("Event fired: " + log);
            console.log("Event fired:" + JSON.stringify(log))
            //fs.appendFile("src/EventTestLog.txt", log+"\n", "utf-8", function(e) {
            fs.appendFile("src/EventTestLog.txt",log['_from']+", "+log["_itemId"]+","+log["quantity"]+', '+log["Price"]+', '+log["add"]+"\n" , "utf-8", function(e){
                if(!e) {
                    console.log(">> Writing to file");
                }
            });
        }
    });

    var value;
    my = await _order.methods.order("0x1111", 3, 100, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000, value:300})
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    my = await _order.methods.order("0x1111", 5, 100, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000, value:500})
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    my = await _order.methods.order("0x1111", 20, 100, "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000,value:2000})
    console.log("---> MyFunction called " + JSON.stringify(my.events.OrderLog.returnValues));
    //process.exit(1);
    
}

doIt();

Overwriting src/OrderEventUse.js


In [7]:
!node src/OrderEventUse.js

- ABI: [object Object],[object Object],[object Object],[object Object]
Account: 0xE91Bf916E93053e0FE4628365afb1Ca16b92D758
Event fired:{"0":"0xE91Bf916E93053e0FE4628365afb1Ca16b92D758","1":"0x1111","2":"3","3":"300","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0xE91Bf916E93053e0FE4628365afb1Ca16b92D758","_itemId":"0x1111","quantity":"3","Price":"300","add":"20 2-gil Hongji-dong Jongro-gu Seoul"}
---> MyFunction called {"0":"0xE91Bf916E93053e0FE4628365afb1Ca16b92D758","1":"0x1111","2":"3","3":"300","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0xE91Bf916E93053e0FE4628365afb1Ca16b92D758","_itemId":"0x1111","quantity":"3","Price":"300","add":"20 2-gil Hongji-dong Jongro-gu Seoul"}
>> Writing to file
Event fired:{"0":"0xE91Bf916E93053e0FE4628365afb1Ca16b92D758","1":"0x1111","2":"5","3":"500","4":"20 2-gil Hongji-dong Jongro-gu Seoul","_from":"0xE91Bf916E93053e0FE4628365afb1Ca16b92D758","_itemId":"0x1111","quantity":"5","Price":"500","add":"20 2-gil Hongji-dong Jongro-gu Seoul"

In [8]:
!cat src/EventTestLog.txt

0xE91Bf916E93053e0FE4628365afb1Ca16b92D758, 0x1111,3, 300, 20 2-gil Hongji-dong Jongro-gu Seoul
0xE91Bf916E93053e0FE4628365afb1Ca16b92D758, 0x1111,5, 500, 20 2-gil Hongji-dong Jongro-gu Seoul
0xE91Bf916E93053e0FE4628365afb1Ca16b92D758, 0x1111,20, 2000, 20 2-gil Hongji-dong Jongro-gu Seoul
